In this chapter we
will discuss a simple, nonlinear model for classification and regression tasks: the
decision tree. We'll use decision trees to build an ad blocker that can learn to classify
images on a web page as banner advertisements or page content. Finally, we will
introduce ensemble learning methods, which combine a set of models to produce an
estimator with better predictive performance than any of its component estimators.

# Decision Trees with scikit-learn

Let's use decision trees to create software that can block banner ads on web pages.
This program will predict whether each of the images on a web page is an
advertisement or article content. Images that are classified as being advertisements
could then be hidden using Cascading Style Sheets. We will train a decision tree
classifier using the Internet Advertisements Data Set from http://archive.ics.uci.edu/ml/datasets/Internet+Advertisements, which contains data for 3,279 images.
The proportions of the classes are skewed; 459 of the images are advertisements and
2,820 are content. Decision tree learning algorithms can produce biased trees from data
with unbalanced class proportions; we will evaluate a model on the unaltered data set
before deciding if it is worth balancing the training data by over- or under-sampling
instances. The explanatory variables are the dimensions of the image, words from the
containing page's URL, words from the image's URL, the image's alt text, the image's
anchor text, and a window of words surrounding the image tag. The response variable
is the image's class. The explanatory variables have already been transformed into
feature representations. The first three features are real numbers that encode the width,
height, and aspect ratio of the images. The remaining features encode binary term
frequencies for the text variables. In the following sample, we will grid search for the
hyperparameter values that produce the decision tree with the greatest accuracy,
and then evaluate the tree's performance on a test set:

In [3]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

if __name__ == '__main__':
    df = pd.read_csv('data/ad.data', header=None)
    explanatory_variable_columns = set(df.columns.values)
    response_variable_column = df[len(df.columns.values)-1]
    
    # The last column describes the targets
    explanatory_variable_columns.remove(len(df.columns.values)-1)
    
    # Encode advertisment as positive class and content as negative class
    y = [1 if e == 'ad.' else 0 for e in response_variable_column]
    X = df[list(explanatory_variable_columns)]
    
    '''
    More than one quarter of the instances are missing at least one of the values
    for the image's dimensions. These missing values are marked by whitespace and a
    question mark. We replaced the missing values with negative one, but we could have
    imputed the missing values; for instance, we could have replaced the missing height
    values with the average height value:
    '''
    X.replace(to_replace=' *\?', value=-1, regex=True, inplace=True)
    
    # Split data into training and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    # set the criterion keyword argument to entropy to build the tree 
    # using the informationgain heuristic
    pipeline = Pipeline([('clf', DecisionTreeClassifier(criterion='entropy'))])
    
    # specify hyperparameters for grid search
    parameters = {
            'clf__max_depth': (150, 155, 160),
            'clf__min_samples_split': (1, 2, 3),
            'clf__min_samples_leaf': (1, 2, 3)
            }
    
    # set GridSearchCV to maximise the models f1 score
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, scoring='f1')
    grid_search.fit(X_train, y_train)
    print 'Best score: %0.3f' % grid_search.best_score_
    print 'Best parameters set:'
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print '\t%s: %r' % (param_name, best_parameters[param_name])
        
    predictions = grid_search.predict(X_test)
    print classification_report(y_test, predictions)
    
    

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fitting 3 folds for each of 27 candidates, totalling 81 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\ProgramData\Anaconda2\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = r'C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

.......................................................

# Tree ensembles

In [4]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

if __name__ == '__main__':
    df = pd.read_csv('data/ad.data', header=None)
    explanatory_variable_columns = set(df.columns.values)
    response_variable_column = df[len(df.columns.values)-1]
    
    # The last column describes the targets
    explanatory_variable_columns.remove(len(df.columns.values)-1)
    
    # Encode advertisment as positive class and content as negative class
    y = [1 if e == 'ad.' else 0 for e in response_variable_column]
    X = df[list(explanatory_variable_columns)]
    
    '''
    More than one quarter of the instances are missing at least one of the values
    for the image's dimensions. These missing values are marked by whitespace and a
    question mark. We replaced the missing values with negative one, but we could have
    imputed the missing values; for instance, we could have replaced the missing height
    values with the average height value:
    '''
    X.replace(to_replace=' *\?', value=-1, regex=True, inplace=True)
    
    # Split data into training and testing set
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    # set the criterion keyword argument to entropy to build the tree 
    # using the informationgain heuristic
    pipeline = Pipeline([('clf', RandomForestClassifier(criterion='entropy'))])
    
    # specify hyperparameters for grid search
    parameters = {
        'clf__n_estimators': (5, 10, 20, 50),
        'clf__max_depth': (50, 150, 250),
        'clf__min_samples_split': (1, 2, 3),
        'clf__min_samples_leaf': (1, 2, 3)
        }
    # set GridSearchCV to maximise the models f1 score
    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, scoring='f1')
    grid_search.fit(X_train, y_train)
    print 'Best score: %0.3f' % grid_search.best_score_
    print 'Best parameters set:'
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print '\t%s: %r' % (param_name, best_parameters[param_name])
        
    predictions = grid_search.predict(X_test)
    print classification_report(y_test, predictions)
    
    

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fitting 3 folds for each of 108 candidates, totalling 324 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\ProgramData\Anaconda2\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    169     pkg_name = mod_name.rpartition('.')[0]
    170     main_globals = sys.modules["__main__"].__dict__
    171     if alter_argv:
    172         sys.argv[0] = fname
    173     return _run_code(code, main_globals, None,
--> 174                      "__main__", fname, loader, pkg_name)
        fname = r'C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py'
        loader = <pkgutil.ImpLoader instance>
        pkg_name = ''
    175 
    176 def run_module(mod_name, init_globals=None,
    177                run_name=None, alter_sys=False):
    178     """Execute a module's code without importing it

.......................................................